# Sek I - Listen

In [1]:
import pandas as pd
import os
import glob


SKRIPT_PATH = os.getcwd()
FILEFOLDER_PATH = ""

# Dataentry variables
ORGANISATION = ""
SPRACHE = ""
FUNKTION = ""
BERUF = ""
STUDIENGANG  =  ""
ORT = ""

In [2]:
os.chdir(FILEFOLDER_PATH)
path = os.getcwd()
xlsx_files = glob.glob(os.path.join(path, "*.xlsx"))


# loop over the list of xlsx files
for file in xlsx_files:

    worksheet = pd.read_excel(file, sheet_name=None, header=None)

    for key in worksheet:
        df = worksheet[key]

        for index, row in df.iterrows():
            if index != df.loc[df[0] == "Name"].index:
                df.drop(index=index, inplace=True)
            elif index == df.loc[df[0] == "Name"].index:
                break

        df = df.reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop([0], axis=0)
        df = df.drop(["Semester"], axis=1)
        df["Organisation"] = ORGANISATION
        df["Sprache"] = SPRACHE
        df["Funktion"] = FUNKTION
        df["Beruf"] = BERUF
        df["Studiengang"] = STUDIENGANG
        df["Ort"] = ORT
        df = df.rename(columns={"Name": "Nachname", "Email-Adresse": "E-Mail"})

        df.to_excel(f"{key}.xlsx", index=False)

os.chdir(SKRIPT_PATH)


# KGPS - Listen

In [1]:
import pandas as pd

In [ ]:
# INPUT

CURRENT_STUDY_YEAR =  # use full first number for study year: 23/24 --> 2023
PATH_STUDENTLIST_RAW = ""
COLUMNS_WANTED = [""]
ORGANISATION = ""
SPRACHE = ""
FUNKTION = ""
BERUF = ""
STUDIENGANG  =  ""
ORT = ""


In [10]:
# PRE-PROCESSING

df_students_raw = pd.read_excel(PATH_STUDENTLIST_RAW)

columns_raw = df_students_raw.columns.to_list()
columns_to_remove = []

for col in columns_raw:
    if col not in (COLUMNS_WANTED):
        columns_to_remove.append(col)


df_students = df_students_raw.drop(columns_to_remove, axis=1)

df_students.rename(columns={"Name": "Nachname",
                   "Mail_privat": "E-Mail"}, inplace=True)

df_students["Organisation"] = ORGANISATION
df_students["Sprache"] = SPRACHE
df_students["Funktion"] = FUNKTION
df_students["Beruf"] = BERUF
df_students["Studiengang"] = STUDIENGANG
df_students["Ort"] = ORT


In [11]:
# PROCESSING (creating new lists)


students_grouped_by_studyyear = df_students.groupby(["Studienjahrgang"])

studyyears = df_students["Studienjahrgang"].dropna().unique().tolist()

for year in studyyears:
    if year > CURRENT_STUDY_YEAR:
        continue

    elif year == CURRENT_STUDY_YEAR:
        df_students_by_studyyear = students_grouped_by_studyyear.get_group(year)
        studyyear_grouped_by_letter = df_students_by_studyyear.groupby(["Lerngruppe"])
        letters = df_students_by_studyyear["Lerngruppe"].dropna().unique().tolist()

        for letter in letters:
            df_students_by_studyyear_and_letter = studyyear_grouped_by_letter.get_group(letter)
            df_students_by_studyyear_and_letter = df_students_by_studyyear_and_letter.reset_index(drop=True)
            
            df_students_by_studyyear_and_letter.to_excel(f"{year}{letter}.xlsx", index=False)

    else:
        df_students_by_studyyear = students_grouped_by_studyyear.get_group(year)
        studyyear_grouped_by_letter = df_students_by_studyyear.groupby(["Lerngruppe klein"])
        letters = df_students_by_studyyear["Lerngruppe klein"].dropna().unique().tolist()

        for letter in letters:
            df_students_by_studyyear_and_letter = studyyear_grouped_by_letter.get_group(letter)
            df_students_by_studyyear_and_letter = df_students_by_studyyear_and_letter.reset_index(drop=True)

            df_students_by_studyyear_and_letter.to_excel(f"{year}{letter}.xlsx", index=False)
